In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
train=pd.read_csv("/kaggle/input/mse2-makeup/train.csv")
test=pd.read_csv("/kaggle/input/mse2-makeup/test.csv")

train.info()
train.isnull().sum()
test.isnull().sum()

test_id=test["id"]
test=test.drop(columns=["id"])

train = train.dropna(subset=['fruit_name'])

train['fruit_name'].shape[0]
train.isnull().sum()

X=train.drop(columns=["fruit_name"])
y=train["fruit_name"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

numeric_features=X.select_dtypes(include=['int64','float64']).columns
categorical_features=X.select_dtypes(include=['object']).columns

numerical_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
categorical_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessing=ColumnTransformer(transformers=[
    ('num',numerical_pipeline,numeric_features),
    ('cat',categorical_pipeline,categorical_features)
])

model=RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

pipeline=Pipeline(steps=[
    ('preprocessing',preprocessing),
    ('model',model)
])

# pipeline.fit(X_train,y_train_enc)
pipeline.fit(X_train,y_train)

# y_proba=pipeline.predict_proba(X_test)
y_pred=pipeline.predict(X_test)

 loss = log_loss(y_test_enc, y_proba)
# print("Log Loss:", loss)
accu=accuracy_score(y_test,y_pred)
print(accu)

# y_final=pipeline.predict_proba(test)
y_final=pipeline.predict(test)

y_final

# class_names = le.classes_  # use label encoder mapping
# submission = pd.DataFrame(y_final, columns=[f"Status_{cls}" for cls in class_names])
# submission.insert(0, 'id', test_id)
# submission.to_csv("submission4.csv", index=False)
# print("\n✅ Submission file created successfully!")
# print(submission.head())
submission=pd.DataFrame({
    'id':test_id,
    'fruit_name':y_final
})
submission.to_csv("submission_final.csv",index=False)